In [1]:
import json

import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
# Season info
season_info = {
    "season_str": ["2021-22", "2020-21", "2018-19", "2017-18", "2016-17", "2015-16"],
    "year": ["2022", "2021", "2019", "2018", "2017", "2016"],
    "tourney_start_date": ["2022-03-15","2021-03-18", "2019-03-19", "2018-03-13", "2017-03-14", "2016-03-15"]
}

In [3]:
# https://www.sportsbookreviewsonline.com/scoresoddsarchives/ncaabasketball/

vegas_dfs = []
for season_year, season_str in zip(season_info["year"], season_info["season_str"]):
    print("Loading Vegas data from", season_year)
    df = pd.read_excel(f"ncaa basketball {season_str}.xlsx", usecols="A:K")
    df["season"] = season_year
    vegas_dfs.append(df)
vegas_df = pd.concat(vegas_dfs)

Loading Vegas data from 2022
Loading Vegas data from 2021
Loading Vegas data from 2019
Loading Vegas data from 2018
Loading Vegas data from 2017
Loading Vegas data from 2016


In [4]:
fte_dfs = []
for year in season_info["year"]:
    print("Downloading FiveThirtyEight data from", year)
    fte_dfs.append(pd.read_csv(f"https://projects.fivethirtyeight.com/march-madness-api/{year}/fivethirtyeight_ncaa_forecasts.csv", parse_dates=["forecast_date"]))
fte_df = pd.concat(fte_dfs)

In [5]:
# Parse date and add year
vegas_df["date_parsed"] = pd.to_datetime(vegas_df["Date"].astype(str).str.zfill(4) + vegas_df["season"].astype(str), format="%m%d%Y")

In [6]:
# Decrement year for games occuring in previous calendar year
vegas_df["date_parsed"] = vegas_df["date_parsed"].mask(vegas_df["date_parsed"].dt.month > 5, vegas_df["date_parsed"] - pd.offsets.DateOffset(years=1))

In [7]:
# Filter out games which are not Tournament games
filtered_dfs = []
for season, tourney_start_date in zip(season_info["year"], season_info["tourney_start_date"]):
    filtered_dfs.append(vegas_df[(vegas_df["season"] == season) & (vegas_df["date_parsed"] >= tourney_start_date)])

vegas_df = pd.concat(filtered_dfs)

In [8]:
# Map names in Vegas dataset to mat
with open("name_map.json", "r") as f:
    name_map = json.load(f)

vegas_df["Team"].replace(name_map, inplace=True)
vegas_df = vegas_df.dropna()

In [9]:
# Merge games into one row and keep only relevant columns
ml_df = vegas_df.iloc[1::2].reset_index().join(vegas_df.iloc[0::2].reset_index(), rsuffix='_2')

In [10]:
# Spread and Total score are mixed together in Open and Close columns, since the spread will always be lower than the total, we can take the min to get the spread
ml_df["spread"] = np.minimum(pd.to_numeric(ml_df["Close"], errors='coerce').fillna(0), pd.to_numeric(ml_df["Close_2"], errors='coerce').fillna(0))
# Spreads are all positive, so we need to use the moneyline to apply the correct sign
ml_df["spread"] = ml_df["spread"].where(ml_df["ML"] > 0, -ml_df["spread"], axis=0)
ml_df = ml_df[["date_parsed", "season", "Team", "Team_2", "ML", "ML_2", "spread", "VH", "VH_2", "Final", "Final_2"]]

In [11]:
ml_df = ml_df.sort_values("date_parsed")

In [12]:
# Keep only Mento_numpyrnament predictions
fte_df = fte_df[fte_df["gender"] == "mens"].sort_values("forecast_date")

In [13]:
def get_538_win_prob(row, fte_df):
    """
    Get win probability for each game using FiveThirtyEight forecasts
    """
    # Get rows with prediction for current team and forecast date before game date
    fte_rows = fte_df[(fte_df["team_name"] == row["Team"]) & (fte_df["forecast_date"] < row["date_parsed"])]
    if len(fte_rows) > 1:
        # Take the latest prediction
        fte_row = fte_rows.iloc[-1]
    else:
        # No prediction is available
        return np.NaN
    
    # Iterate through rd1-rd7 probabilities
    # The win probability for a given game is equivalent to the probability of advancing to the next round
    for i in range(1, 8):
        if ((fte_row[f"rd{i}_win"] < 1.0) and (fte_row[f"rd{i}_win"] > 0.0)):
            return fte_row[f"rd{i}_win"]
    
    # No probability was found
    return np.NaN

In [14]:
ml_df["FiveThirtyEight Win Prob"] = ml_df.apply(get_538_win_prob, fte_df=fte_df, axis=1)
ml_df["fte_spread_raw"] = 400/30.464 * np.log10(1/ml_df["FiveThirtyEight Win Prob"] - 1)
ml_df["FiveThirtyEight Spread"] = round(ml_df["fte_spread_raw"] * 2) / 2

In [15]:
ml_df = ml_df.dropna()

In [16]:
# helper function to convert moneylines into probabilties
def ml_to_prob(ml):
    if ml > 0:
        return 100 / (100 + ml)
    else:
        ml_pos = ml * -1
        return ml_pos / (100 + ml_pos)

In [17]:
ml_df["ml_prob1"] = ml_df["ML"].apply(ml_to_prob)
ml_df["ml_prob2"] = ml_df["ML_2"].apply(ml_to_prob)

# adjust moneyline probabilities for juice/vig
ml_df["ml_prob_adj1"] = ml_df.ml_prob1 / (ml_df.ml_prob1 + ml_df.ml_prob2)
ml_df["ml_prob_adj2"] = ml_df.ml_prob2 / (ml_df.ml_prob1 + ml_df.ml_prob2)

In [18]:
# create ground truth column, return 1 if team1 won and 0 if they lost
ml_df["team1_win"] = (ml_df.Final > ml_df.Final_2).astype(int)

# ground truth column for spreads
# make sure we subtract in the correct order to follow convention of favored teams having negative spreads
ml_df["point_diff"] = (ml_df.Final_2 - ml_df.Final).astype(int)

In [19]:
ml_df["team1_cover"] = (ml_df.Final + ml_df.spread > ml_df.Final_2).astype(int)
print(f"Vegas Team 1 Cover Rate: {ml_df.team1_cover.mean():.4%}")
print(f"FiveThirtyEight Team 1 Cover Rate: {(ml_df.Final + ml_df['FiveThirtyEight Spread'] > ml_df.Final_2).mean():.4%}")

Vegas Team 1 Cover Rate: 45.1282%
FiveThirtyEight Team 1 Cover Rate: 45.8974%


In [20]:
# Compute Brier scores to compare models
from sklearn.metrics import brier_score_loss, mean_squared_error

def get_brier_scores(df):
    fte_brier = brier_score_loss(df.team1_win, df["FiveThirtyEight Win Prob"])
    vegas_brier = brier_score_loss(df.team1_win, df.ml_prob_adj1)
    fte_skill = 1 - fte_brier / vegas_brier
    
    return fte_brier, vegas_brier, fte_skill


def get_mse(df):
    fte_mse = mean_squared_error(df.point_diff, df["FiveThirtyEight Spread"])
    vegas_mse = mean_squared_error(df.point_diff, df.spread)
    fte_skill = 1 - fte_mse / vegas_mse
    
    return fte_mse, vegas_mse, fte_skill

In [21]:
fte_brier, vegas_brier, fte_skill = get_brier_scores(ml_df)
fte_mse, vegas_mse, fte_mse_skill = get_mse(ml_df)
results = [
    {
        "season": "All Seasons",
        "FiveThirtyEight Brier score": fte_brier,
        "Vegas Moneyline Brier score": vegas_brier,
        "FiveThirtyEight Skill score over Vegas (probabilities)": fte_skill,
        "FiveThirtyEight MSE": fte_mse,
        "Vegas Spread MSE": vegas_mse,
        "FiveThirtyEight Skill score over Vegas (spreads)": fte_mse_skill
    }
]
for season in season_info["year"]:
    fte_brier, vegas_brier, fte_skill = get_brier_scores(ml_df[ml_df["season"] == season])
    fte_mse, vegas_mse, fte_mse_skill = get_mse(ml_df[ml_df["season"] == season])
    results.append({
        "season": season,
        "FiveThirtyEight Brier score": fte_brier,
        "Vegas Moneyline Brier score": vegas_brier,
        "FiveThirtyEight Skill score over Vegas (probabilities)": fte_skill,
        "FiveThirtyEight MSE": fte_mse,
        "Vegas Spread MSE": vegas_mse,
        "FiveThirtyEight Skill score over Vegas (spreads)": fte_mse_skill
    })

pd.DataFrame(results).style.format({
    "FiveThirtyEight Skill score over Vegas (probabilities)": '{:+,.4%}',
    "FiveThirtyEight Skill score over Vegas (spreads)": '{:+,.4%}'
})

,season,FiveThirtyEight Brier score,Vegas Moneyline Brier score,FiveThirtyEight Skill score over Vegas (probabilities),FiveThirtyEight MSE,Vegas Spread MSE,FiveThirtyEight Skill score over Vegas (spreads)
0,All Seasons,0.191028,0.191346,+0.1662%,137.642949,135.906410,-1.2777%
1,2022,0.209117,0.210692,+0.7476%,148.806818,146.371212,-1.6640%
2,2021,0.220430,0.214666,-2.6850%,152.757692,150.342308,-1.6066%
3,2019,0.158121,0.161184,+1.9003%,133.333333,135.746212,+1.7775%
4,2018,0.206192,0.205400,-0.3858%,151.863636,148.681818,-2.1400%
5,2017,0.174317,0.170528,-2.2217%,109.280769,106.630769,-2.4852%
6,2016,0.177354,0.185275,+4.2755%,129.096774,126.895161,-1.7350%


# Wagering Strategy Testing

In [22]:
spread_odds = 1.9

ml_df["ml_dec1"] = 1 / ml_df.ml_prob1
ml_df["ml_dec2"] = 1 / ml_df.ml_prob2
# ml_df["fte_cover_prob"] = 1 / (1+10**(30.464/400*(ml_df.fte_spread_raw - ml_df.spread)))

ml_df["fte_cover_prob"] = 1 / (1+np.exp(0.16065354*(ml_df.fte_spread_raw - ml_df.spread)))


ml_df["spread_dec1"] = spread_odds
ml_df["spread_dec2"] = spread_odds

In [23]:
ml_df = ml_df.sort_values("date_parsed")

In [24]:
# Constant params for wager logic
BANKROLL = 100
MAX_BET_FRAC = 0.15  # Limiting the max wager helps prevent large losses
MIN_PROFIT_FRAC = 0.05
MAX_SPREAD = 18 # Don't bet on overly large spreads


def kelly_criterion(prob, odds_dec):
    """
    Helper method to compute Kelly Criterion.
    Given the estimated true probability of an event occurring and the decimal odds of the event,
        returns the fraction of your bankroll you should bet.
    If odds are not favorable, returns 0.
    """
    return max(prob - ((1 - prob) / (odds_dec - 1)), 0)


def log_exp(prob, odds_dec, amt, bankroll):
    """
    Helper method to compute Log Expected Wealth.
    """
    return np.log(prob * (bankroll + amt * (odds_dec - 1)) + (1 - prob) * (bankroll - amt))


def wager_logic(date, team1, team2, point_diff, spread, ml_dec1, ml_dec2, fte_prob, spread_dec1, spread_dec2, fte_cover_prob, strategy, bankroll):
    """
    Method to choose how much to wager (if at all).
    
    params:
    team1, team2: names of teams
    point_diff: team 2 score - team 1 score
    spread: vegas spread
    ml_dec1 and ml_dec2: Vegas moneyline odds for team 1 and 2 respectively in decimal form.
    fte_prob1 and fte_prob2: FiveThirtyEight's predicted win probability for each team
    spread_dec1 and spread_dec2: Vegas spread odds in decimal form.
    fte_cover_prob1 and fte_cover_prob2: FiveThirtyEight's predicted cover probability (computed above)
    strategy:
        - "ML" - only bet moneylines
        - "Spread" - only bet spreads
        - "Best" - choose the more optimal of the two on a per bet basis
    bankroll: bankroll available for this bet
    
    returns a dict with the following keys:
        wager_description - string describing what wager was chosen, i.e. "Kansas ML" or "Kansas -3.5"
        wager_amount
        wager_type - 'spread' or 'moneyline'
        wager_won
        bankroll - updated after accounting for bet won or lost
    """
    
    ml1_amt = min(kelly_criterion(fte_prob, ml_dec1), MAX_BET_FRAC) * bankroll
    ml2_amt = min(kelly_criterion(1 - fte_prob, ml_dec2), MAX_BET_FRAC) * bankroll
    
    spread1_amt = min(kelly_criterion(fte_cover_prob, spread_dec1), MAX_BET_FRAC) * bankroll
    spread2_amt = min(kelly_criterion(1 - fte_cover_prob, spread_dec2), MAX_BET_FRAC) * bankroll
    
    if strategy == "ML" or abs(spread) > MAX_SPREAD:
        spread1_amt = 0
        spread2_amt = 0
    elif strategy == "Spread":
        ml1_amt = 0
        ml2_amt = 0
    elif strategy == "Prefer ML":
        # We want to bet moneylines if they're positive, and fall back on spreads if they're not
        if max(ml1_amt, ml2_amt) > 0:
            # Moneyline bet is positive, let's not bet spread
            spread1_amt = 0
            spread2_amt = 0
    elif strategy == "Prefer Spread":
        # We want to bet spreads if they're positive, and fall back on moneylines if they're not
        if max(spread1_amt, spread2_amt) > 0:
            # Spread bet is positive, let's not bet moneyline
            ml1_amt = 0
            ml2_amt = 0
    
    i = np.argmax([
        log_exp(fte_prob, ml_dec1, ml1_amt, bankroll),
        log_exp(1 - fte_prob, ml_dec2, ml2_amt, bankroll),
        log_exp(fte_cover_prob, spread_dec1, spread1_amt, bankroll),
        log_exp(1 - fte_cover_prob, spread_dec2, spread2_amt, bankroll),
        np.log(bankroll)
    ])
    
    if i == 0:
        # Wager on Team 1 Moneyline
        wager_type = 'moneyline'
        wager_description = team1 + " " + wager_type
        wager_amount = ml1_amt
        wager_won = point_diff < 0
        wager_odds = ml_dec1
    elif i == 1:
        # Wager on Team 2 Moneyline
        wager_type = 'moneyline'
        wager_description = team2 + " " + wager_type
        wager_amount = ml2_amt
        wager_won = point_diff > 0
        wager_odds = ml_dec2
    elif i == 2:
        # Wager on Team 1 Spread
        wager_type = 'spread'
        wager_description = f"{team1} {spread:+.1f}"
        wager_amount = spread1_amt
        wager_won = point_diff < spread
        wager_odds = spread_dec1
    elif i == 3:
        # Wager on Team 2 Spread
        wager_type = 'spread'
        wager_description = f"{team2} {-spread:+.1f}"
        wager_amount = spread2_amt
        wager_won = point_diff > spread
        wager_odds = spread_dec2
    elif i == 4:
        # No wager recommended
        wager_type = ""
        wager_description = ""
        wager_amount = 0
        wager_won = np.nan
        wager_odds = 0
        
    potential_profit = wager_amount * (wager_odds - 1)
    
    if (potential_profit / bankroll) < MIN_PROFIT_FRAC:
        # Not enough profit, do not wager
        wager_type = ""
        wager_description = ""
        wager_amount = 0
        wager_won = np.nan
        wager_odds = 0
    elif wager_won:
        bankroll += potential_profit
    else:
        bankroll -= wager_amount
    
    return {
        "date_parsed": date, 
        "Team": team1,
        "Team_2": team2,
        "wager_desc": wager_description,
        "wager_type": wager_type,
        "wager_amount": wager_amount,
        "wager_won": wager_won,
        "bankroll": bankroll
    }

In [25]:
def track_bankroll(df, strategy="Both"):
    wager_df = [
        {
            "date_parsed": np.nan,
            "Team": "",
            "Team_2": "",
            "wager_desc": "Start",
            "wager_type": "",
            "wager_amount": 0,
            "wager_won": np.nan,
            "bankroll": BANKROLL
        }
    ]
    prev_bankroll = BANKROLL
    for i in range(0, df.shape[0]):
        row = df.loc[df.index[i]]
        
        wager_logic_output = wager_logic(
            row.date_parsed, row.Team, row.Team_2, row.point_diff, row.spread,
            row.ml_dec1, row.ml_dec2, row['FiveThirtyEight Win Prob'],
            row.spread_dec1, row.spread_dec2, row.fte_cover_prob,
            strategy=strategy,
            bankroll=prev_bankroll
        )
    
        wager_df.append(wager_logic_output)
        prev_bankroll = wager_logic_output['bankroll']

    return pd.DataFrame(wager_df)

In [26]:
results = []
for season in season_info["year"]:
    wager_df = track_bankroll(ml_df[ml_df["season"] == season], strategy="ML")
    spread_df = track_bankroll(ml_df[ml_df["season"] == season], strategy="Spread")
    both_df = track_bankroll(ml_df[ml_df["season"] == season], strategy="Both")
    prefer_spread_df = track_bankroll(ml_df[ml_df["season"] == season], strategy="Prefer Spread")
    prefer_ml_df = track_bankroll(ml_df[ml_df["season"] == season], strategy="Prefer ML")
    results.append(
        {
            "season": season,
            "Final Profit (MLs only)": wager_df['bankroll'].iloc[-1] - BANKROLL,
            "Final Profit (Spreads only)": spread_df['bankroll'].iloc[-1] - BANKROLL,
            "Final Profit (Both ML and Spread)": both_df['bankroll'].iloc[-1] - BANKROLL,
            "Final Profit (Prefer Spread)": prefer_spread_df['bankroll'].iloc[-1] - BANKROLL,
            "Final Profit (Prefer ML)": prefer_ml_df['bankroll'].iloc[-1] - BANKROLL,
        }
    )
results_df = pd.DataFrame(results)

In [31]:
results_df

,season,Final Profit (MLs only),Final Profit (Spreads only),Final Profit (Both ML and Spread),Final Profit (Prefer Spread),Final Profit (Prefer ML)
0,2022,18.148593,-19.429151,-9.468179,-27.244951,30.661558
1,2021,157.433773,62.880228,55.929751,83.760667,153.812271
2,2019,9.453441,-8.675295,-28.523927,-16.216939,-4.593311
3,2018,-3.197554,45.811989,43.164413,54.400168,-2.520456
4,2017,-33.914641,-22.495324,-41.809775,-0.269733,-37.944251
5,2016,109.985035,9.164463,89.086285,25.142555,94.856766


In [41]:
season = "2017"
games_df = ml_df[ml_df["season"] == season][["date_parsed", "Team", "Team_2", "spread", "Final", "Final_2", "FiveThirtyEight Win Prob", "FiveThirtyEight Spread", "ml_dec1", "ml_dec2"]]
wagers_df = track_bankroll(ml_df[ml_df["season"] == season], strategy="Both")

In [42]:
games_df.merge(wagers_df)

,date_parsed,Team,Team_2,spread,Final,Final_2,FiveThirtyEight Win Prob,FiveThirtyEight Spread,ml_dec1,ml_dec2,wager_desc,wager_type,wager_amount,wager_won,bankroll
0,2017-03-15,UC-Davis,North Carolina Central,3.5,67.0,63.0,0.370940,3.0,2.500000,1.588235,,,0.000000,NaN,100.000000
1,2017-03-15,Providence,Southern California,2.0,71.0,75.0,0.582211,-2.0,2.150000,1.740741,Providence moneyline,moneyline,15.000000,False,85.000000
2,2017-03-16,Virginia,North Carolina-Wilmington,-7.5,76.0,71.0,0.844767,-9.5,1.285714,3.900000,Virginia -7.5,spread,10.695066,False,74.304934
3,2017-03-16,Notre Dame,Princeton,-6.0,60.0,58.0,0.754708,-6.5,1.363636,3.350000,,,0.000000,NaN,74.304934
4,2017-03-16,West Virginia,Bucknell,-13.0,86.0,80.0,0.901830,-12.5,1.090909,8.000000,,,0.000000,NaN,74.304934
5,2017-03-16,Florida State,Florida Gulf Coast,-11.0,86.0,80.0,0.875459,-11.0,1.125000,6.500000,,,0.000000,NaN,74.304934
6,2017-03-16,Maryland,Xavier,-2.0,65.0,76.0,0.505718,-0.0,1.769231,2.100000,Xavier +2.0,spread,7.562939,True,81.111580
7,2017-03-16,Florida,ETennesseeSt,-9.5,80.0,65.0,0.874918,-11.0,1.200000,5.000000,,,0.000000,NaN,81.111580
8,2017-03-16,Purdue,Vermont,-9.0,80.0,70.0,0.859696,-10.5,1.243902,4.400000,,,0.000000,NaN,81.111580
9,2017-03-16,Wisconsin,Virginia Tech,-6.0,84.0,74.0,0.789798,-7.5,1.400000,3.100000,Wisconsin moneyline,moneyline,12.166737,True,85.978274
